## 일일 박스오피스 데이터 & 영화정보 merge

- 응답구조
    - boxofficeType :문자열	박스오피스 종류
    - showRange : 	문자열	박스오피스 조회 일자를 출력
    - rnum	문자열	순번을 출력
    - rank	문자열	해당일자의 박스오피스 순위를 출력
    - rankInten	문자열	전일대비 순위의 증감분을 출력
    - rankOldAndNew	문자열	랭킹에 신규진입여부를 출력
    - “OLD” : 기존 , “NEW” : 신규
    - movieCd: 문자열	영화의 대표코드를 출력
    - movieNm : 문자열	영화명(국문)을 출력
    - openDt : 문자열	영화의 개봉일을 출력
    - salesAmt : 문자열	해당일의 매출액을 출력
    - salesShare : 문자열	해당일자 상영작의 매출총액 대비 해당 영화의 매출비율을 출력
    - salesInten : 문자열	전일 대비 매출액 증감분을 출력
    - salesChange : 문자열	전일 대비 매출액 증감 비율을 출력
    - salesAcc : 문자열	누적매출액을 출력
    - audiCnt : 문자열	해당일의 관객수를 출력
    - audiInten : 문자열	전일 대비 관객수 증감분을 출력
    - audiChange : 문자열	전일 대비 관객수 증감 비율을 출력
    - audiAcc : 문자열	누적관객수를 출력합니다.
    - scrnCnt : 문자열	해당일자에 상영한 스크린수를 출력
    - showCnt : 문자열	해당일자에 상영된 횟수를 출력

In [84]:
import pandas as pd
import numpy as np
import time

df = pd.read_csv('../data/boxoffice_20_23.csv')
data = pd.read_csv('../data/boxoffice_15_19.csv')

In [85]:
concat_df = pd.concat([data,df], axis = 0)

In [86]:
df = concat_df.copy()

In [87]:
df['current_date']

0        2015-01-01
1        2015-01-01
2        2015-01-01
3        2015-01-01
4        2015-01-01
            ...    
14605    2023-12-31
14606    2023-12-31
14607    2023-12-31
14608    2023-12-31
14609    2023-12-31
Name: current_date, Length: 32870, dtype: object

In [88]:
movie_codes = df['movieCd'].drop_duplicates().tolist()

In [171]:
df24 = pd.read_csv('../data/boxoffice_24.csv')

### 영화 상세정보 크롤링

In [9]:
# import requests
# import pandas as pd

# key = 'b713496d3bb846528153ba2985029132'
# movieCd = 20182530
# url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movieCd}'
# response = requests.get(url)
# r_data = response.json()

In [122]:
%%time
import requests

key = '75b69dc273dc0e8cef4e9cfb4736799f'
movie_codes = df24['movieCd'].drop_duplicates().tolist()

info_dfs = []

for movieCd in movie_codes:
    try:
        url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movieCd}'
        response = requests.get(url)
        r_data = response.json()

        # 필요한 필드 선택
        selected_data = {
            'movieCd': r_data['movieInfoResult']['movieInfo'].get('movieCd', None),
            'movieNm': r_data['movieInfoResult']['movieInfo'].get('movieNm', None),
            'showTm': r_data['movieInfoResult']['movieInfo'].get('showTm', None),
            'prdtYear': r_data['movieInfoResult']['movieInfo'].get('prdtYear', None),
            'openDt': r_data['movieInfoResult']['movieInfo'].get('openDt', None),
            'prdtStatNm': r_data['movieInfoResult']['movieInfo'].get('prdtStatNm', None),
            'typeNm': r_data['movieInfoResult']['movieInfo'].get('typeNm', None),
            'nationNm': r_data['movieInfoResult']['movieInfo']['nations'][0]['nationNm'] if r_data['movieInfoResult']['movieInfo'].get('nations') else None,
            'genreNm': r_data['movieInfoResult']['movieInfo']['genres'][0]['genreNm'] if r_data['movieInfoResult']['movieInfo'].get('genres') else None,
            'director': r_data['movieInfoResult']['movieInfo']['directors'][0]['peopleNm'] if r_data['movieInfoResult']['movieInfo'].get('directors') else None,
            'actor': ', '.join(actor['peopleNm'] for actor in r_data['movieInfoResult']['movieInfo'].get('actors', [])),
            'showTypeGroupNm': ', '.join(stype['showTypeGroupNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
            'showTypeNm': ', '.join(stype['showTypeNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
            'audits': ', '.join(audit['watchGradeNm'] for audit in r_data['movieInfoResult']['movieInfo'].get('audits', [])) if r_data['movieInfoResult']['movieInfo'].get('audits') else None
        }

        info_df = pd.DataFrame([selected_data])
        info_dfs.append(info_df)
    except Exception as e:
        print(f"Error occurred while processing movie code {movieCd}: {e}")

total_info_df = pd.concat(info_dfs, ignore_index=True)
total_info_df.head()


CPU times: user 360 ms, sys: 177 ms, total: 537 ms
Wall time: 39 s


,movieCd,movieNm,showTm,prdtYear,openDt,prdtStatNm,typeNm,nationNm,genreNm,director,actor,showTypeGroupNm,showTypeNm,audits
0,20203702,노량: 죽음의 바다,152,2022,20231220,개봉,장편,한국,사극,김한민,"김윤석, 백윤식, 정재영, 허준호, 김성규, 이규형, 이무생, 최덕문, 안보현, 박...","2D, 2D, 2D, 4D, IMAX, ScreenX, DOLBYCINEMA","디지털, 디지털 한글자막 CC, 디지털 가치봄, 4D, IMAX, ScreenX, ...",12세이상관람가
1,20212866,서울의 봄,141,2022,20231122,개봉,장편,한국,드라마,김성수,"황정민, 정우성, 이성민, 박해준, 김성균, 김의성, 정동환, 안내상, 유성주, 최...","2D, 2D, 2D, IMAX","디지털, 디지털 한글자막 CC, 디지털 가치봄, IMAX",12세이상관람가
2,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,94,2023,20231222,개봉,장편,일본,애니메이션,오네 히토시,,"2D, 2D, 2D","디지털, 디지털 한글자막 CC, 디지털 더빙",전체관람가
3,20235735,아쿠아맨과 로스트 킹덤,123,2023,20231220,개봉,장편,미국,액션,제임스 완,"제이슨 모모아, 패트릭 윌슨, 야히아 압둘 마틴 2세, 앰버 허드, 니콜 키드먼","2D, 4D, IMAX, ScreenX, DOLBYCINEMA","디지털, 4D, IMAX, ScreenX, DOLBYCINEMA",12세이상관람가
4,20235596,트롤: 밴드 투게더,91,2023,20231220,개봉,장편,미국,애니메이션,월트 도른,"안나 켄드릭, 저스틴 팀버레이크, 주이 디샤넬, 에이미 슈머, 앤드류 라넬스","2D, 2D, 4D, 4D, DOLBYCINEMA","디지털, 디지털 더빙, 4D 더빙, 4D, DOLBYCINEMA",전체관람가


In [172]:
total_info_df.to_csv('../data/boxoffice_info_24.csv',index = False, encoding = 'utf-8-sig')

In [155]:
# total_info_df = pd.read_csv('../data/boxoffice_info_15_23.csv')

In [173]:
df24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rnum           910 non-null    int64  
 1   rank           910 non-null    int64  
 2   rankInten      910 non-null    int64  
 3   rankOldAndNew  910 non-null    object 
 4   movieCd        910 non-null    int64  
 5   movieNm        910 non-null    object 
 6   openDt         910 non-null    object 
 7   salesAmt       910 non-null    int64  
 8   salesShare     910 non-null    float64
 9   salesInten     910 non-null    int64  
 10  salesChange    910 non-null    float64
 11  salesAcc       910 non-null    int64  
 12  audiCnt        910 non-null    int64  
 13  audiInten      910 non-null    int64  
 14  audiChange     910 non-null    float64
 15  audiAcc        910 non-null    int64  
 16  scrnCnt        910 non-null    int64  
 17  showCnt        910 non-null    int64  
 18  current_da

In [174]:
total_info_df['movieCd'] = total_info_df['movieCd'].astype(int)

In [175]:
total24 = pd.merge(df24,total_info_df, on= ['movieCd','movieNm'], how = 'left')

In [176]:
total24[['openDt_x','openDt_y']]

,openDt_x,openDt_y
0,2023-12-20,20231220
1,2023-11-22,20231122
2,2023-12-22,20231222
3,2023-12-20,20231220
4,2023-12-20,20231220
...,...,...
905,2024-03-27,20240327
906,2024-01-31,20240131
907,2024-04-03,20240403
908,2024-03-27,20240327


In [177]:
# 날짜 형식으로 변환
total24['openDt_y'] = pd.to_datetime(total24['openDt_y'], format='%Y%m%d')
total24['openDt_x'].fillna(total24['openDt_y'], inplace=True)
total24.drop(columns = 'openDt_y', inplace = True)
total24.rename(columns = {'openDt_x' : 'openDt'}, inplace = True)

In [178]:
total24.to_csv('../data/boxoffice_24_total.csv',index = False, encoding = 'utf-8-sig')

In [179]:
total24.current_date

0      2024-01-01
1      2024-01-01
2      2024-01-01
3      2024-01-01
4      2024-01-01
          ...    
905    2024-03-31
906    2024-03-31
907    2024-03-31
908    2024-03-31
909    2024-03-31
Name: current_date, Length: 910, dtype: object

In [180]:
df = pd.read_csv('../data/boxoffice_15_23_total.csv')
df24 = pd.read_csv('../data/boxoffice_24_total.csv')

/var/folders/nz/vk3v33x95bs4kf3ymy0z0my00000gn/T/ipykernel_24082/1278547656.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/boxoffice_15_23_total.csv')


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33523 entries, 0 to 33522
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rnum             33523 non-null  int64  
 1   rank             33523 non-null  int64  
 2   rankInten        33523 non-null  int64  
 3   rankOldAndNew    33523 non-null  object 
 4   movieCd          33523 non-null  object 
 5   movieNm          33523 non-null  object 
 6   openDt           33523 non-null  object 
 7   salesAmt         33523 non-null  int64  
 8   salesShare       33523 non-null  float64
 9   salesInten       33523 non-null  int64  
 10  salesChange      33523 non-null  float64
 11  salesAcc         33523 non-null  int64  
 12  audiCnt          33523 non-null  int64  
 13  audiInten        33523 non-null  int64  
 14  audiChange       33523 non-null  float64
 15  audiAcc          33523 non-null  int64  
 16  scrnCnt          33523 non-null  int64  
 17  showCnt     

In [182]:
df24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rnum             910 non-null    int64  
 1   rank             910 non-null    int64  
 2   rankInten        910 non-null    int64  
 3   rankOldAndNew    910 non-null    object 
 4   movieCd          910 non-null    int64  
 5   movieNm          910 non-null    object 
 6   openDt           910 non-null    object 
 7   salesAmt         910 non-null    int64  
 8   salesShare       910 non-null    float64
 9   salesInten       910 non-null    int64  
 10  salesChange      910 non-null    float64
 11  salesAcc         910 non-null    int64  
 12  audiCnt          910 non-null    int64  
 13  audiInten        910 non-null    int64  
 14  audiChange       910 non-null    float64
 15  audiAcc          910 non-null    int64  
 16  scrnCnt          910 non-null    int64  
 17  showCnt         

In [183]:
total_24 = pd.concat([df, df24], axis = 0)

In [185]:
total_24.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34433 entries, 0 to 909
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rnum             34433 non-null  int64  
 1   rank             34433 non-null  int64  
 2   rankInten        34433 non-null  int64  
 3   rankOldAndNew    34433 non-null  object 
 4   movieCd          34433 non-null  object 
 5   movieNm          34433 non-null  object 
 6   openDt           34433 non-null  object 
 7   salesAmt         34433 non-null  int64  
 8   salesShare       34433 non-null  float64
 9   salesInten       34433 non-null  int64  
 10  salesChange      34433 non-null  float64
 11  salesAcc         34433 non-null  int64  
 12  audiCnt          34433 non-null  int64  
 13  audiInten        34433 non-null  int64  
 14  audiChange       34433 non-null  float64
 15  audiAcc          34433 non-null  int64  
 16  scrnCnt          34433 non-null  int64  
 17  showCnt       

In [186]:
total_24.to_csv('../data/boxoffice_15_24_total.csv',index = False, encoding = 'utf-8-sig')